# SK-1-Fundamentals : Introduction a Semantic Kernel

**Navigation** : [Index](README.md) | [02-Functions >>](02-SemanticKernel-Advanced.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Installer et configurer **Semantic Kernel** pour Python
2. Comprendre le role du **Kernel** comme orchestrateur central
3. Ajouter des **services LLM** (OpenAI, Azure, Hugging Face)
4. Charger et utiliser des **plugins** (prompts templates)
5. Creer des **fonctions semantiques inline**
6. Gerer un **chat conversationnel** avec historique

### Prerequis

- Python 3.10+
- Cle API OpenAI (ou Azure OpenAI)
- Fichier `.env` configure

### Duree estimee : 45 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Installation | pip install semantic-kernel |
| 2 | Configuration | .env, Kernel, Services |
| 3 | Premier Kernel | Initialisation, service LLM |
| 4 | Plugins | Chargement depuis fichiers |
| 5 | Fonctions inline | Prompts dynamiques |
| 6 | Chat | Historique, KernelArguments |

> **Qu'est-ce que Semantic Kernel ?** Un SDK Microsoft open-source pour integrer des LLMs dans vos applications. Il orchestre les appels aux modeles, gere les plugins, et permet de creer des agents intelligents.

In [1]:
# Installation de Semantic Kernel (si nécessaire)
%pip install semantic-kernel

# Vérification de la version installée
from semantic_kernel import __version__
print(f"Semantic Kernel version : {__version__}")


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


Semantic Kernel version : 1.39.3



### 📌 **Importation des bibliothèques nécessaires**
Dans cette cellule, nous allons importer les modules principaux.


In [2]:
import os
import json
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments  # Correction de l'import


## **📁 2. Chargement des paramètres de configuration**
### 📌 **Lecture des paramètres depuis un fichier `.env` ou JSON**
Le fichier de configuration `.env` doit contenir les clés nécessaires pour accéder aux services OpenAI/Azure OpenAI.

💡 **Vérifiez que vous avez bien créé un fichier `.env`** dans le même dossier que ce notebook avec ces valeurs :

```plaintext
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_CHAT_MODEL_ID="gpt-4o-mini"
```

👉 Nous allons maintenant **charger ces paramètres en Python** :

In [3]:
# Chargement du fichier .env
load_dotenv()

# Récupération des clés API et du modèle
llm_service = os.getenv("GLOBAL_LLM_SERVICE", "OpenAI")
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

# Vérification
print(f"Service sélectionné : {llm_service}")
print(f"Modèle utilisé : {model_id}")


Service sélectionné : OpenAI
Modèle utilisé : gpt-4o


In [4]:
# Importer le Kernel depuis Semantic Kernel
from semantic_kernel import Kernel

# Créer une instance du Kernel
kernel = Kernel()
print("Kernel initialisé avec succès.")


Kernel initialisé avec succès.


In [5]:
import os
from dotenv import load_dotenv
from openai import AsyncOpenAI
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# Récupère la valeur du .env et nettoie les espaces
base_url_from_env = os.getenv("OPENAI_BASE_URL", "").strip()

# Détermine explicitement l'URL finale
if base_url_from_env:
    final_base_url = base_url_from_env
    print(f"Utilisation de l'URL du .env : {final_base_url}")
else:
    final_base_url = "https://api.openai.com/v1" # <<< URL par défaut explicite
    print(f"Utilisation de l'URL par défaut : {final_base_url}")

model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")
print(f"Utilisation du modèle : {model_id}")
print(f"Clé API utilisée : {'Oui' if api_key else 'Non'}")

async def test_connection():
    try:
        if not api_key:
            print("ERREUR : Clé API non définie.")
            return

        # Utilise final_base_url déterminé ci-dessus
        client = AsyncOpenAI(api_key=api_key, base_url=final_base_url)
        response = await client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": "Say hello!"}],
            max_tokens=10,
            timeout=20
        )
        print("Connexion réussie !")
        print("Réponse :", response.choices[0].message.content)
    except Exception as e:
        print(f"ERREUR lors du test de connexion : {type(e).__name__} - {e}")
        import traceback
        traceback.print_exc()

# Exécuter le test
await test_connection()

Utilisation de l'URL par défaut : https://api.openai.com/v1
Utilisation du modèle : gpt-4o
Clé API utilisée : Oui


Connexion réussie !
Réponse : Hello! How can I assist you today?


## Configuration du service LLM

Nous devons connecter notre Kernel à un service de Chat Completion.  
Pour cet exemple, nous allons utiliser OpenAI. Si vous préférez Azure OpenAI, adaptez le code en conséquence (voir la documentation).

La configuration se fait via l'ajout d'un service au Kernel.  
Assurez-vous que votre fichier `.env` contient votre clé API.


In [6]:
# Importation du service OpenAI pour le Chat Completion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Ajout du service "default" au Kernel
kernel.add_service(OpenAIChatCompletion(service_id="default"))
print("Service OpenAI ajouté au Kernel.")


Service OpenAI ajouté au Kernel.


### Interpretation : Architecture du Kernel

Le Kernel est le **coeur** de Semantic Kernel. Il orchestre :

| Composant | Role | Exemple |
|-----------|------|---------|
| **Services** | Connexions aux LLMs | OpenAIChatCompletion, AzureChatCompletion |
| **Plugins** | Collections de fonctions | FunPlugin, WriterPlugin |
| **Fonctions** | Unites d'execution | Joke, Summarize, Chat |
| **Arguments** | Parametres dynamiques | KernelArguments(input=...) |

```
┌─────────────────────────────────────────┐
│                 Kernel                  │
│  ┌─────────────┐  ┌─────────────────┐  │
│  │  Services   │  │    Plugins      │  │
│  │  (LLMs)     │  │  (Functions)    │  │
│  └─────────────┘  └─────────────────┘  │
│           ↓              ↓              │
│       ┌─────────────────────────┐      │
│       │     invoke(func, args)  │      │
│       └─────────────────────────┘      │
└─────────────────────────────────────────┘
```

> **Service ID** : Chaque service a un identifiant unique. "default" est utilise si non specifie lors de l'invocation.

## Utiliser un Plugin de Prompt

Semantic Kernel permet de charger des **prompt plugins** stockés sur disque.  
Dans cet exemple, nous chargerons le plugin "FunPlugin" qui contient, par exemple, une fonction pour générer une blague.

Les fichiers du plugin (le prompt et sa configuration) sont stockés dans le répertoire `prompt_template_samples/`.  
Nous allons charger ce plugin et invoquer la fonction "Joke" pour générer une blague sur un sujet donné.


In [7]:
# Chemin correct vers les plugins
plugins_directory = "./prompt_template_samples/"

# Vérifier si le dossier du plugin existe avant de charger
if os.path.exists(os.path.join(plugins_directory, "FunPlugin")):
    fun_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")
    joke_function = fun_plugin["Joke"]
    print("Plugin chargé avec succès.")
else:
    print("⚠️ Le plugin FunPlugin est introuvable. Vérifiez le chemin et assurez-vous qu'il est bien présent.")


# Invoquer la fonction pour générer une blague sur un thème donné
# Pour ce faire, nous utilisons des KernelArguments (ici, seul l'input est nécessaire)
from semantic_kernel.functions import KernelArguments

print(joke_function)

# Exemple : générer une blague sur "time travel to dinosaur age" avec un style "super silly"
joke_response = await kernel.invoke(joke_function, KernelArguments(input="time travel to dinosaur age", style="super silly"))
print("Blague générée :", joke_response)


Plugin chargé avec succès.
metadata=KernelFunctionMetadata(name='Joke', plugin_name='FunPlugin', description='Generate a funny joke', parameters=[KernelParameterMetadata(name='input', description='Joke subject', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'Joke subject'}, include_in_function_choices=True), KernelParameterMetadata(name='style', description='Give a hint about the desired joke style', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'Give a hint about the desired joke style'}, include_in_function_choices=True)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None) invocation_duration_histogram=<opentelemetry.metr

Blague générée : Why did the time traveler bring a suitcase full of sandwiches to the dinosaur age?

Because he heard the T-Rex had a "roaring" appetite, and he didn't want to be the main course! So, he figured he'd distract it with a prehistoric picnic. But when he got there, the T-Rex just looked at the sandwiches, then at him, and said, "I’m on a paleo diet, but thanks for the offer!"


## Définir une fonction sémantique en ligne

Outre l'utilisation de plugins stockés sur disque, il est possible de définir des fonctions sémantiques directement dans votre code Python.  
Cette approche est particulièrement utile pour :
- Générer dynamiquement des prompts en fonction du contexte
- Prototyper rapidement des idées sans créer de fichiers séparés

Dans cet exemple, nous allons créer une fonction qui résume un texte donné en quelques mots (TL;DR).


In [8]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

# Définition du prompt
tldr_prompt = """
{{$input}}

Donne-moi un résumé en 5 mots ou moins.
"""

# Configuration de l'exécution
execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
    max_tokens=2000,
    temperature=0.7,
)

# Configuration du prompt template
tldr_template_config = PromptTemplateConfig(
    template=tldr_prompt,
    name="tldr",
    template_format="semantic-kernel",
    input_variables=[InputVariable(name="input", description="Texte à résumer", is_required=True)],
    execution_settings=execution_settings,
)

# Ajout de la fonction au Kernel
tldr_function = kernel.add_function(function_name="tldrFunction", plugin_name="tldrPlugin", prompt_template_config=tldr_template_config)

# Exécution de la fonction
async def run_tldr():
    input_text = "Demo était une poétesse grecque ancienne connue pour un unique poème gravé sur le Colosse de Memnon."
    tldr_summary = await kernel.invoke(tldr_function, KernelArguments(input=input_text))
    print("Résumé (TL;DR) :", tldr_summary)

# Lancer la fonction
await run_tldr()


Résumé (TL;DR) : Poétesse grecque, unique poème gravé.


## Chat interactif avec le Kernel

Semantic Kernel offre également la possibilité de créer des interactions de type chatbot.  
Nous allons configurer une fonction de chat qui utilise des **Kernel Arguments** pour conserver l'historique de la conversation.

L'objectif est de simuler une conversation où l'utilisateur envoie un message, le bot y répond, et l'historique est mis à jour à chaque échange.


In [9]:
from semantic_kernel.contents import ChatHistory

# Initialiser l'historique
chat_history = ChatHistory()
chat_history.add_system_message("Vous êtes un chatbot utile et vous fournissez des recommandations de livres.")

# Définition du prompt de chat
chat_prompt = """
{{$history}}
User: {{$user_input}}
ChatBot:
"""

# Configuration de l'exécution
chat_exec_settings = OpenAIChatPromptExecutionSettings(
    service_id="default",
    ai_model_id=model_id,
    max_tokens=2000,
    temperature=0.7,
)

# Configuration du chat template
chat_template_config = PromptTemplateConfig(
    template=chat_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="Message de l'utilisateur", is_required=True),
        InputVariable(name="history", description="Historique de la conversation", is_required=True),
    ],
    execution_settings=chat_exec_settings,
)

# Ajout au Kernel
chat_function = kernel.add_function(function_name="chat", plugin_name="chatPlugin", prompt_template_config=chat_template_config)

# Fonction asynchrone pour le chat
async def chat(input_text: str):
    print(f"Utilisateur : {input_text}")
    response = await kernel.invoke(chat_function, KernelArguments(
        user_input=input_text, 
        history=str(chat_history),
        allow_dangerously_set_content=True
    ))
    print(f"ChatBot : {response}")
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(response))

# Exécution des exemples
await chat("Salut, je cherche des suggestions de livres sur la philosophie antique.")
await chat("Peux-tu m'en recommander quelques-uns ?")


Utilisateur : Salut, je cherche des suggestions de livres sur la philosophie antique.


ChatBot : Salut ! La philosophie antique est un domaine fascinant, et il y a plusieurs ouvrages qui pourraient t'intéresser :

1. **"La République" de Platon** - Un dialogue sur la justice, la politique et l'organisation de la société. C'est l'une des œuvres les plus célèbres de Platon.

2. **"Éthique à Nicomaque" d'Aristote** - Un traité sur la vertu et le chemin vers une vie éthique et épanouie.

3. **"Les Méditations" de Marc Aurèle** - Des réflexions personnelles de l'empereur romain stoïque, offrant des conseils sur la résilience et la sagesse.

4. **"Ennéades" de Plotin** - Une collection de six groupes de neuf traités qui explore la métaphysique, l'éthique et la théologie.

5. **"Les Pensées" d'Épictète** (souvent publiées sous le titre "Manuel d'Épictète") - Un guide pratique pour mener une vie stoïque.

6. **"Les Discours" de Plutarque** - Bien que Plutarque soit plus connu pour ses "Vies parallèles", ses essais philosophiques sont également très influents.

Chacun de ces livr

ChatBot : Bien sûr ! Voici quelques recommandations supplémentaires sur la philosophie antique qui pourraient t'intéresser :

1. **"Le Banquet" de Platon** - Un dialogue centré sur le thème de l'amour, où différents personnages discutent de sa nature et de son rôle dans la vie humaine.

2. **"Penseurs grecs avant Socrate" de G.S. Kirk, J.E. Raven et M. Schofield** - Un recueil qui explore les idées des premiers philosophes grecs, souvent appelés les présocratiques.

3. **"Les Stoïciens" de Pierre Hadot** - Une introduction accessible à la philosophie stoïque, mettant en lumière ses concepts clés et son influence durable.

4. **"Les Lettres à Lucilius" de Sénèque** - Une série de lettres philosophiques qui abordent divers sujets, allant de l'éthique personnelle à la gestion des émotions.

5. **"Le Phédon" de Platon** - Un dialogue qui traite de l'âme et de l'immortalité, racontant les derniers moments de Socrate avant sa mort.

6. **"Fragments" d'Héraclite** - Bien que peu de ses écrits

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **Kernel** | Orchestrateur central SK | `Kernel()` |
| **Service** | Connexion LLM | `kernel.add_service(OpenAIChatCompletion(...))` |
| **Plugin** | Collection de fonctions | `kernel.add_plugin(parent_directory=..., plugin_name=...)` |
| **Fonction semantique** | Prompt template execute | `kernel.add_function(prompt_template_config=...)` |
| **KernelArguments** | Parametres dynamiques | `KernelArguments(input=..., style=...)` |
| **ChatHistory** | Historique conversation | `ChatHistory()` + `add_user_message()` |

## Points cles a retenir

1. **Le Kernel est le coeur de SK** - Il orchestre services, plugins et fonctions
2. **Les plugins sont modulaires** - Chargeables depuis fichiers ou definis inline
3. **KernelArguments permet le passage dynamique** - Variables injectees dans les templates
4. **ChatHistory preserve le contexte** - Essentiel pour les conversations multi-tours

## Pour aller plus loin

| Notebook suivant | Contenu |
|-----------------|---------|
| [02-Functions](02-SemanticKernel-Advanced.ipynb) | Function Calling moderne, Memory, Groundedness |
| [03-Agents](03-SemanticKernel-Agents.ipynb) | ChatCompletionAgent, AgentGroupChat |

---

**Navigation** : [Index](README.md) | [02-Functions >>](02-SemanticKernel-Advanced.ipynb)